In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

2022-11-16 12:07:54.405636: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 12:07:54.541293: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-16 12:07:54.541315: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-16 12:07:54.569223: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 12:07:56.077459: W tensorflow/stream_executor/platform/de

In [3]:
CLASSES5 = ['angry', 'happy', 'neutral', 'sad', 'surprise']
CLASSES6 = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']
CLASSES7 = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [4]:
# Load the model.
model = hub.load('https://tfhub.dev/google/yamnet/1')

2022-11-16 12:08:00.120360: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 12:08:00.120613: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-16 12:08:00.120698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-16 12:08:00.120749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-16 12:08:00.120796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [5]:
df = pd.read_csv('../Datasets/custom_db/df.csv')
df['file'] = df['file'].apply(lambda x: '../Datasets/custom_db/' + x[2:])
df

,file,emotion,split,dataset,augmentation,language
0,../Datasets/custom_db/tess/train/angry_1068.wav,angry,train,tess,none,en
1,../Datasets/custom_db/tess/train/surprise_1286...,surprise,train,tess,none,en
2,../Datasets/custom_db/tess/train/fear_248.wav,fear,train,tess,none,en
3,../Datasets/custom_db/tess/train/happy_2790.wav,happy,train,tess,none,en
4,../Datasets/custom_db/tess/train/surprise_139.wav,surprise,train,tess,none,en
...,...,...,...,...,...,...
92777,../Datasets/custom_db/bser/augment/happy_1017_...,happy,augment,bser,freqmask,bn
92778,../Datasets/custom_db/bser/augment/happy_660_p...,happy,augment,bser,pitchshift,bn
92779,../Datasets/custom_db/bser/augment/neutral_479...,neutral,augment,bser,freqmask,bn
92780,../Datasets/custom_db/bser/augment/happy_841_t...,happy,augment,bser,timemask,bn


In [6]:
def get_all():
    df = pd.read_csv('../Datasets/custom_db/df.csv')
    df['file'] = df['file'].apply(lambda x: '../Datasets/custom_db/' + x[2:])
    return df

In [7]:
def get_df(ds):
    df = get_all()
    df = df[df['dataset'] == ds]
    return df

In [8]:
def get_lang(lang):
    df = get_all()
    df = df[df['language'] == lang]
    return df

In [9]:
def get_embeddings(filename, label):
    data, sr = librosa.load(filename, sr=None, mono=True, res_type='kaiser_best')

    if sr != 16000:
        raise ValueError('Sample rate is not 16kHz of file {}'.format(filename))

    _, embeddings, _ = model(data)
    num_embeddings = tf.shape(embeddings)[0]
    return embeddings, tf.repeat(label, num_embeddings)

In [1]:
def save_embeddings(df, filename, split):
    features = []
    labels = []

    if df['emotion'].nunique() == 6:
        classes = CLASSES6
    elif df['emotion'].nunique() == 5:
        classes = CLASSES5
    else:
        classes = CLASSES7

    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        embs, label = get_embeddings(row['path'], row['emotion'])
        assert(len(embs) == len(label))
        for i in range(embs.shape[0]):
            features.append(embs[i])
            labels.append(classes.index(label[i]))
    
    features = np.array(features)
    labels = np.array(labels)
    np.save(filename + split + '_X.npy', features)
    np.save(filename + split + '_y.npy', labels)

In [ ]:
df = get_df('tess')
train = df[df['split'] == 'train']
train_aug = df[(df['split'] == 'train') | (df['split'] == 'augment')]
val = df[df['split'] == 'val']
test = df[df['split'] == 'test']
save_embeddings(train, './features/yamnet_emb/tess/', 'train')
save_embeddings(train_aug, './features/yamnet_emb/tess/', 'train_aug')
save_embeddings(val, './features/yamnet_emb/tess/', 'val')
save_embeddings(test, './features/yamnet_emb/tess/', 'test')

In [ ]:
df = get_df('savee')
train = df[df['split'] == 'train']
train_aug = df[(df['split'] == 'train') | (df['split'] == 'augment')]
val = df[df['split'] == 'val']
test = df[df['split'] == 'test']
save_embeddings(train, './features/yamnet_emb/savee/', 'train')
save_embeddings(train_aug, './features/yamnet_emb/savee/', 'train_aug')
save_embeddings(val, './features/yamnet_emb/savee/', 'val')
save_embeddings(test, './features/yamnet_emb/savee/', 'test')

# Testing

In [ ]:
# X = np.load('./features/vggish_emb/savee/X.npy', allow_pickle=True)
# y = np.load('./features/vggish_emb/savee/y.npy', allow_pickle=True)

In [ ]:
# X.shape, y.shape

In [ ]:
# X[200]

In [ ]:
# y[200]

In [ ]:
stop

# Spectogram

In [ ]:
from math import ceil


def plot_embeddings(df, name):

    if df['Emotion'].nunique() == 6:
        classes = CLASSES6
    else:
        classes = CLASSES7

    fig, axs = plt.subplots(2, ceil(len(classes) / 2), figsize=(15, 6), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .3, wspace=.3)
    fig.suptitle(f'{name} Yamnet Embeddings')
    fig.tight_layout()
    axs = axs.ravel()

    for i in range(len(classes)):
        emb,_,log_mel = get_embeddings(df[df['Emotion'] == classes[i]]['File_Path'].iloc[0], df[df['Emotion'] == classes[i]]['Emotion'].iloc[0])
        # axs[i].imshow(log_mel.numpy().T, aspect='auto', interpolation='nearest', origin='lower', cmap='jet')
        axs[i].plot(emb[0])
        axs[i].set_title(classes[i])

def plot_mel(df, name):

    if df['Emotion'].nunique() == 6:
        classes = CLASSES6
    else:
        classes = CLASSES7

    fig, axs = plt.subplots(2, ceil(len(classes) / 2), figsize=(15, 6), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .3, wspace=.3)
    fig.suptitle(f'{name} Yamnet Log Mel Spectrogram')
    fig.tight_layout()
    axs = axs.ravel()

    for i in range(len(classes)):
        emb,_,log_mel = get_embeddings(df[df['Emotion'] == classes[i]]['File_Path'].iloc[0], df[df['Emotion'] == classes[i]]['Emotion'].iloc[0])
        axs[i].imshow(log_mel.numpy().T, aspect='auto', interpolation='nearest', origin='lower', cmap='jet')
        axs[i].set_title(classes[i])

In [ ]:
plot_embeddings(tess, 'Tess')
plot_mel(tess, 'Tess')

In [ ]:
plot_embeddings(savee, 'Savee')
plot_mel(savee, 'Savee')

In [ ]:
plot_embeddings(ravdess, 'Ravdess')
plot_mel(ravdess, 'Ravdess')

In [ ]:
plot_embeddings(crema, 'Crema')
plot_mel(crema, 'Crema')